In [ ]:
from rich import traceback

traceback.install()

In [ ]:
with open("possible_cases_4.txt") as f:
    lines = f.readlines()

# examples
# Sequence: [(0, 1), (0, 2), (0, 3), (1, 2), (1, 2, 3)]
# Coverage history: [{(0, 1)}, {(0, 2)}, {(0, 3)}, {(1, 2)}, {(2, 3), (1, 3)}]

# Sequence: [(0, 1), (0, 2), (0, 3), (1, 2), (0, 1, 2, 3)]
# Coverage history: [{(0, 1)}, {(0, 2)}, {(0, 3)}, {(1, 2)}, {(2, 3), (1, 3)}]

# Sequence: [(0, 1), (0, 2), (0, 3), (1, 3), (1, 2, 3)]
# Coverage history: [{(0, 1)}, {(0, 2)}, {(0, 3)}, {(1, 3)}, {(2, 3), (1, 2)}]

sequence_list = []
coverage_list = []
for line in lines:
    line = line.strip()
    if line.startswith("Sequence"):
        sequence = eval(line.split(":")[1])
        sequence_list.append(sequence)
    if line.startswith("Coverage history"):
        coverage = eval(line.split(":")[1])
        coverage_list.append(coverage)

In [ ]:
print(len(sequence_list))
print(len(coverage_list))

- There are eleven possible 4-motifs in total (allowing isolated nodes):
    - {0}(0-1) An empty graph
    - {1}(1-1) A single edge
    - {2}(2-1) Two disconnected edges
    - {3}(2-2) Two connected edges (i.e., wedge or open triangle)
    - {4}(3-1) A 3-path
    - {5}(3-2) A triangle
    - {6}(3-3) A 3-star
    - {7}(4-1) A 4-cycle
    - {8}(4-2) A triangle + an edge
    - {9}(5-1) Two triangles sharing a common edge
    - {10}(6-1) A 4-clique

In [ ]:
# convert each "coverage" to an accumulated coverage
accumulated_coverage_list = []
for coverage in coverage_list:
    accumulated_coverage = [frozenset()]
    for cov in coverage:
        accumulated_coverage.append(accumulated_coverage[-1] | cov)
    accumulated_coverage_list.append(accumulated_coverage)

In [ ]:
from collections import defaultdict
from itertools import combinations, chain
import torch

device = torch.device("cuda:1")

all_pairs = list(combinations(range(4), 2))  # in total 6 pairs
all_pairs = [frozenset(pair) for pair in all_pairs]

# https://stackoverflow.com/questions/1482308
def powerset(iterable):
    "powerset([1,2,3]) --> () (1,) (2,) (3,) (1,2) (1,3) (2,3) (1,2,3)"
    s = list(iterable)
    return chain.from_iterable(combinations(s, r) for r in range(len(s)+1))

all_node_subsets = [frozenset(subset) for subset in powerset(range(4))]
nodeSubset2pairs = dict()
for node_subset in all_node_subsets:
    nodeSubset2pairs[node_subset] = frozenset([frozenset(pair_) for pair_ in combinations(node_subset, 2)])

all_pairs_subsets = [frozenset(subset) for subset in powerset(all_pairs)]  # in total 2^6 = 64 subsets

# if the pairs in {key} have been covered, sampling the nodes in {value} together is useless
# i.e., the pairs between the nodes in {value} have been covered
coveredParis2uselessSubset = defaultdict(list)
for pair_subset in all_pairs_subsets:
    for node_subset, pairs_set in nodeSubset2pairs.items():
        if pairs_set.issubset(pair_subset):  # all the pairs between the nodes in node_subset have been covered
            coveredParis2uselessSubset[pair_subset].append(node_subset)


In [ ]:
motif2pairs = defaultdict(list)

# - There are eleven possible 4-motifs in total (allowing isolated nodes):
#     - {0}(0-1) An empty graph
#     - {1}(1-1) A single edge
#     - {2}(2-1) Two disconnected edges
#     - {3}(2-2) Two connected edges (i.e., wedge or open triangle)
#     - {4}(3-1) A 3-path
#     - {5}(3-2) A triangle
#     - {6}(3-3) A 3-star
#     - {7}(4-1) A 4-cycle
#     - {8}(4-2) A triangle + an edge
#     - {9}(5-1) Two triangles sharing a common edge
#     - {10}(6-1) A 4-clique

# 0. An empty graph
motif2pairs[0] = [frozenset()]

# 1. A single edge
for pair in all_pairs:
    motif2pairs[1].append(frozenset([pair]))

# 2. Two disconnected edges
# 3. Two connected edges (i.e., wedge or open triangle)

for pair1, pair2 in combinations(all_pairs, 2):
    if len(pair1 & pair2) == 0:
        motif2pairs[2].append(frozenset([pair1, pair2]))
    else:
        motif2pairs[3].append(frozenset([pair1, pair2]))

# 4. A 3-path
# first choose the two end points
for v_start, v_end in combinations(range(4), 2):
    remaining_nodes = set(range(4)) - {v_start, v_end}
    for v_mid in remaining_nodes:
        v_other = remaining_nodes - {v_mid}
        v_other = v_other.pop()
        # path: v_start -> v_mid -> v_other -> v_end
        motif2pairs[4].append(frozenset([frozenset([v_start, v_mid]), frozenset([v_mid, v_other]), frozenset([v_other, v_end])]))

# 5. A triangle
for v1, v2, v3 in combinations(range(4), 3):
    motif2pairs[5].append(frozenset([frozenset([v1, v2]), frozenset([v2, v3]), frozenset([v3, v1])]))

# 6. A 3-star
for center in range(4):
    v1, v2, v3 = set(range(4)) - {center}
    motif2pairs[6].append(frozenset([frozenset([center, v1]), frozenset([center, v2]), frozenset([center, v3])]))

# 7. A 4-cycle
# choose the node opposite to node 0
for v_oppo in range(1, 4):
    # cycle: 0 -> v1 -> v_oppo -> v2 -> 0
    remaining_nodes = set(range(4)) - {0, v_oppo}
    v1, v2 = remaining_nodes
    motif2pairs[7].append(frozenset([frozenset([0, v1]), frozenset([v1, v_oppo]), frozenset([v_oppo, v2]), frozenset([v2, 0])]))

# 8. A triangle + an edge
for v1, v2, v3 in combinations(range(4), 3):
    remaining_nodes = set(range(4)) - {v1, v2, v3}
    v4 = remaining_nodes.pop()
    motif2pairs[8].append(frozenset([frozenset([v1, v2]), frozenset([v2, v3]), frozenset([v3, v1]), frozenset([v1, v4])]))
    motif2pairs[8].append(frozenset([frozenset([v1, v2]), frozenset([v2, v3]), frozenset([v3, v1]), frozenset([v2, v4])]))
    motif2pairs[8].append(frozenset([frozenset([v1, v2]), frozenset([v2, v3]), frozenset([v3, v1]), frozenset([v3, v4])]))

# 9. Two triangles sharing a common edge
# choose the common edge
for v1, v2 in combinations(range(4), 2):
    remaining_nodes = set(range(4)) - {v1, v2}
    v3, v4 = remaining_nodes
    # edges: v1-v2, v1-v3, v1-v4, v2-v3, v2-v4
    motif2pairs[9].append(frozenset([frozenset([v1, v2]), frozenset([v1, v3]), frozenset([v1, v4]), frozenset([v2, v3]), frozenset([v2, v4])]))

# 10. A 4-clique
motif2pairs[10] = [frozenset(all_pairs)]

# pairs2motif: for each combination of pairs, find the corresponding motif
pairs2motif = dict()
for motif, pairs_list in motif2pairs.items():
    for pairs in pairs_list:
        pairs2motif[pairs] = motif


In [ ]:
from itertools import product

from tqdm import tqdm

EPS = 1e-6

def compute_pg_dict(g_nodes):
    # the probability that a specific subset of nodes is sampled together in each round
    pg_dict = {}
    pg_dict[frozenset()] = (1 - g_nodes).prod()
    pg_dict[frozenset([0])] = g_nodes[0] * ((1 - g_nodes[[1, 2, 3]]).prod())
    pg_dict[frozenset([1])] = g_nodes[1] * ((1 - g_nodes[[0, 2, 3]]).prod())
    pg_dict[frozenset([2])] = g_nodes[2] * ((1 - g_nodes[[0, 1, 3]]).prod())
    pg_dict[frozenset([3])] = g_nodes[3] * ((1 - g_nodes[[0, 1, 2]]).prod())
    pg_dict[frozenset([0, 1])] = g_nodes[0] * g_nodes[1] * (1 - g_nodes[[2, 3]]).prod()
    pg_dict[frozenset([0, 2])] = g_nodes[0] * g_nodes[2] * (1 - g_nodes[[1, 3]]).prod()
    pg_dict[frozenset([0, 3])] = g_nodes[0] * g_nodes[3] * (1 - g_nodes[[1, 2]]).prod()
    pg_dict[frozenset([1, 2])] = g_nodes[1] * g_nodes[2] * (1 - g_nodes[[0, 3]]).prod()
    pg_dict[frozenset([1, 3])] = g_nodes[1] * g_nodes[3] * (1 - g_nodes[[0, 2]]).prod()
    pg_dict[frozenset([2, 3])] = g_nodes[2] * g_nodes[3] * (1 - g_nodes[[0, 1]]).prod()
    pg_dict[frozenset([0, 1, 2])] = g_nodes[[0, 1, 2]].prod() * (1 - g_nodes[3])
    pg_dict[frozenset([0, 1, 3])] = g_nodes[[0, 1, 3]].prod() * (1 - g_nodes[2])
    pg_dict[frozenset([0, 2, 3])] = g_nodes[[0, 2, 3]].prod() * (1 - g_nodes[1])
    pg_dict[frozenset([1, 2, 3])] = g_nodes[[1, 2, 3]].prod() * (1 - g_nodes[0])
    pg_dict[frozenset([0, 1, 2, 3])] = g_nodes.prod()
    
    return pg_dict

def sample_motif_probs(seq, cov, cov_acc, p_nodes, pg_dict, R):
    # seq: a sampling sequence
    # cov: an accumulated coverage history
    # p_nodes: a 2d tensor of shape (4, 4)    
    # pg_dict: pg_dict[node_subset] = Pr[node_subset are sampled together in each round]
    # R: an integer scalar
    
    # in R rounds, {seq} is a subsequence of the whole length-R sampling sequence
    len_seq = len(seq)
    n_remain_slots = R - len_seq
    
    # there are (len_seq + 1) possible positions to insert the remaining sampling rounds
    # and those remaining rounds should be "useless" according to the accumulated coverage history
    
    # now, based on the accumulated coverage history {cov}, compute the probability of useless subsets
    assert len(cov_acc) == len_seq + 1
    sequence_prob = 1.0
    for node_subset in seq:
        node_subset = frozenset(node_subset)
        sequence_prob *= pg_dict[node_subset]
    useless_probs = torch.zeros(len_seq + 1, device=device)
    for i in range(len_seq + 1):
        covered_pairs_before_i = cov_acc[i]
        covered_pairs_before_i = frozenset([frozenset(pair) for pair in covered_pairs_before_i])
        useless_subsets_before_i = coveredParis2uselessSubset[covered_pairs_before_i]
        useless_probs[i] = sum(pg_dict[node_subset] for node_subset in useless_subsets_before_i)            
    
    # the probability of the sequence is sequence_prob * {STAR}
    # {STAR} = \sum_{n1 + n2 + ... + n_{len_seq + 1} = n_remain_slots} \prod_{i=1}^{len_seq + 1} useless_probs[i]^{n_i}        
    # Using generating function -->
    # {STAR} = \sum_{i} useless_probs[i]^{n_remain_slots} / (\prod_{j \neq i} (1 - useless_probs[j] / useless_probs[i]))
    star = 0.0
    for i in range(len_seq + 1):
        num_i = useless_probs[i] ** n_remain_slots
        den_i = 1.0
        for j in range(len_seq + 1):
            if j != i:
                den_i *= (1 - useless_probs[j] / (useless_probs[i] + EPS))
        star += num_i / den_i
    
    prob_seq = sequence_prob * star
    
    # compute the motif probabilities
    motif_probs = torch.zeros(11, device=device)
    
    # for each subset of pairs in the coverage {cov}, compute the probability of results
    pair2prob = {}
    pair2prob[frozenset([0, 1])] = p_nodes[0, 1]
    pair2prob[frozenset([0, 2])] = p_nodes[0, 2]
    pair2prob[frozenset([0, 3])] = p_nodes[0, 3]
    pair2prob[frozenset([1, 2])] = p_nodes[1, 2]
    pair2prob[frozenset([1, 3])] = p_nodes[1, 3]
    pair2prob[frozenset([2, 3])] = p_nodes[2, 3]
    
    pairs2prob_list = [dict() for _ in cov]
    
    for i_, cov_ in enumerate(cov):        
        pairs_in_cov = [frozenset(pair) for pair in cov_]
        probs_in_cov = [pair2prob[pair] for pair in pairs_in_cov]
        
        # sort the pairs w.r.t their probabilities, also get the corresponding probabilities (small to large)
        sorted_probs_in_cov, sorted_pairs_in_cov = zip(*sorted(zip(probs_in_cov, pairs_in_cov), reverse=False))
        
        for j_ in range(len(sorted_probs_in_cov)):
            pairs_from_j = sorted_pairs_in_cov[j_:]
            prob_from_j = sorted_probs_in_cov[j_] if j_ == 0 else sorted_probs_in_cov[j_] - sorted_probs_in_cov[j_ - 1]
            pairs2prob_list[i_][frozenset(pairs_from_j)] = prob_from_j
        pairs2prob_list[i_][frozenset()] = 1.0 - sorted_probs_in_cov[-1]
        
    # now consider the combinations
    for pairs_comb in product(*pairs2prob_list):
        pairs_comb = [frozenset(pairs) for pairs in pairs_comb]
        probs_comb = [pairs2prob_list[i_][pairs] for i_, pairs in enumerate(pairs_comb)]
        prob_comb = torch.prod(torch.tensor(probs_comb, device=device))
        total_pairs_comb = frozenset().union(*pairs_comb)                
        i_motif = pairs2motif[total_pairs_comb]
        motif_probs[i_motif] += prob_comb        
    
    return prob_seq, motif_probs

def motif_probs_indep(p_nodes):
    # given the probabilities of each pair, compute the motif probabilities when all the pairs are independent
    motif_probs = torch.zeros(11, device=device)
    
    pair2prob = {}
    pair2prob[frozenset([0, 1])] = p_nodes[0, 1]
    pair2prob[frozenset([0, 2])] = p_nodes[0, 2]
    pair2prob[frozenset([0, 3])] = p_nodes[0, 3]
    pair2prob[frozenset([1, 2])] = p_nodes[1, 2]
    pair2prob[frozenset([1, 3])] = p_nodes[1, 3]
    pair2prob[frozenset([2, 3])] = p_nodes[2, 3]       
    
    for pair_subset in all_pairs_subsets:
        pairs_not_exist = frozenset(all_pairs) - pair_subset
        prob_ = 1.0
        for pair in pair_subset:
            prob_ *= pair2prob[pair]
        for pair in pairs_not_exist:
            prob_ *= (1 - pair2prob[pair])
        i_motif = pairs2motif[pair_subset]
        motif_probs[i_motif] += prob_

    return motif_probs


def total_motif_probs(seq_list, cov_list, cov_acc_list, p, g, R):
    # seq_list: a list of node sampling sequences
    # cov_list: a list of coverage histories
    # cov_acc_list: a list of accumulated coverage histories
    # p: a 2d tensor of shape (n, n)
    # g: a 1d tensor of shape (n)
    # R: an integer scalar

    n_nodes = p.size(0)
    assert p.size(1) == n_nodes
    assert g.size(0) == n_nodes

    nodes = torch.arange(n_nodes, device=device)

    # there are 11 possible motifs
    motif_num = torch.zeros(11, device=device)

    # for each subset of nodes of size 4
    for nodes in tqdm(torch.combinations(nodes, 4), desc="Total motifs"):
        remaining_sample_probs = 1.0
        p_nodes = p[nodes][:, nodes]
        g_nodes = g[nodes]
        # for each possible way of covering the nodes
        for seq, cov, cov_acc in tqdm(zip(seq_list, cov_list, cov_acc_list), desc="Covering ways", total=len(seq_list)):
            pg_dict = compute_pg_dict(g_nodes)
            # compute the probability of this specific way of covering and the corresponding motif probabilities
            sample_probs_i, motif_probs_i = sample_motif_probs(
                seq, cov, cov_acc, p_nodes, pg_dict, R
            )
            motif_num += motif_probs_i * sample_probs_i
            remaining_sample_probs -= sample_probs_i        
        motif_num += motif_probs_indep(p_nodes) * remaining_sample_probs
    
    return motif_num

In [ ]:
n = 10

# a random edge probability matrix
p = torch.rand(n, n, device=device)
p = (p + p.t()) / 2
# clean the diagonal
p.fill_diagonal_(0.0)

# a random node sampling probability
g = torch.rand(n, device=device)

R = 100_000

total_motif_probs(sequence_list, coverage_list, accumulated_coverage_list, p, g, R)